In [14]:
import os
from PIL import Image
from albumentations.pytorch import ToTensorV2
import albumentations as A
from torch import nn
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset
import numpy as np
import glob
import cv2
import imageio
import pandas as pd
import os
from torch.utils.data import DataLoader
import torch
import torchvision.transforms as T
from torchvision.models import resnet18, resnet50
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output
import warnings

from torch.optim import Adam
from sklearn.model_selection import train_test_split

In [15]:
class NatureDataset(Dataset):

    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        pic = self.df.iloc[index]["pic"]
        ans = self.df.iloc[index]["answers"]
        # print(pic)
        pic = np.array(Image.open(pic).convert("RGB"))
        if self.transform is not None:
            augmentations = self.transform(image = pic)
            pic = augmentations

        return pic["image"], ans

In [31]:
df = pd.DataFrame()
# кликун - 0, малой -1 , шипун - 2
path1 = r"C:\Users\ga232\Природа\root\klikun\images"
path2 = r"C:\Users\ga232\Природа\root\Maliy\images"
path3 = r"C:\Users\ga232\Природа\root\Shipun\images"


df["pic"] = os.listdir(path1)[:3000] + os.listdir(path2)[:3000] + os.listdir(path3)[:3000]
answers = np.append(np.array([0 for i in range(3000)]),np.array([1 for i in range(3000)]))
answers = np.append(answers,np.array([2 for i in range(3000)]))
df["answers"] = answers

#Справа тоже впиши свой путь
df["pic"].iloc[:3000] = df["pic"].iloc[:3000].apply(lambda x: f"C://Users//ga232//Природа//root//klikun//images//{x}" )
df["pic"].iloc[3000:6000] = df["pic"].iloc[3000:6000].apply(lambda x: f"C://Users//ga232//Природа//root//Maliy//images//{x}" )
df["pic"].iloc[6000:] = df["pic"].iloc[6000:].apply(lambda x: f"C://Users//ga232//Природа//root//Shipun//images//{x}" )

C:\Users\ga232\AppData\Local\Temp\ipykernel_10448\1901198503.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["pic"].iloc[:3000] = df["pic"].iloc[:3000].apply(lambda x: f"C://Users//ga232//Природа//root//klikun//images//{x}" )
C:\Users\ga232\AppData\Local\Temp\ipykernel_10448\1901198503.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["pic"].iloc[3000:6000] = df["pic"].iloc[3000:6000].apply(lambda x: f"C://Users//ga232//Природа//root//Maliy//images//{x}" )
C:\Users\ga232\AppData\Local\Temp\ipykernel_10448\1901198503.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [32]:
train, test = train_test_split(df, test_size=0.2)
train = df # обучался на всём датасете
train_transform = A.Compose(
    [

        A.Resize(224, 224),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        ToTensorV2(),
    ]
)

test_transform = A.Compose(
    [
        A.Resize(224, 224),
        ToTensorV2(),

    ]
)

In [33]:
train_dataset = NatureDataset(train, transform=train_transform)
test_dataset = NatureDataset(test, transform=test_transform)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, pin_memory=True)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [34]:
x,y = next(iter(test_loader))

In [35]:
model = resnet50(pretrained=True)



model.fc = nn.Linear(in_features=2048, out_features=3, bias=True)

model = model.to(device)

model.load_state_dict(torch.load("Final_model.pt"))
model.eval()
#print(evaluate(model1, test_loader))


C:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [36]:
pred = torch.max(model(x.type(torch.float32).to(device)),1)[1]

In [37]:
sum(pred == y.to(device))/len(y)

tensor(1., device='cuda:0')